In [13]:
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time

In [14]:
# Flipkart Tablets Search URL
BASE_URL = "https://www.flipkart.com/search?q=tablets&page={}"

# Headers to mimic a real browser
HEADERS = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.0.0 Safari/537.36"
}

In [15]:
def scrape_page(page_num, data):
    url = BASE_URL.format(page_num)
    
    try:
        response = requests.get(url, headers=HEADERS, timeout=10)

        # Debugging: Check response code and content
        print(f"\nFetching: {url}")
        print(f"Response Code: {response.status_code}")

        if response.status_code != 200:
            print(f"Failed to fetch page {page_num}, Status Code: {response.status_code}")
            print("Response Content (first 500 chars):", response.text[:500])
            return

        # Parse the page content
        soup = BeautifulSoup(response.text, "html.parser")

        # Find all product containers
        products = soup.find_all("a", class_="CGtC98")

        if not products:
            print(f"No products found on page {page_num}. Flipkart might be blocking the request.")
            return
        
        for product in products:
            try:
                name = product.find("div", class_="KzDlHZ")
                name = name.text.strip() if name else "Unknown"

                price = product.find("div", class_="Nx9bqj _4b5DiR")
                price = price.text.strip() if price else None
                
                Ori_Price = product.find("div", class_= "yRaY8j ZYYwLA")
                Ori_Price = Ori_Price.text.strip() if Ori_Price else None
                
                rating = product.find("div", class_="XQDdHH")
                rating = rating.text.strip() if rating else None

                reviews = product.find("span", class_="Wphh3N")
                reviews = reviews.text.split("&")[0].strip() if reviews else None
                
                Discount = product.find("div", class_="UkUFwK")
                Discount = Discount.text.strip() if Discount else None 

                # Extract product specifications
                specs = product.find_all("ul", class_="G4BRas")
                brand, storage, ram, processor, battery, feature, display, camera = None, None, None, None, None, None, None, None

                for spec in specs:
                    items = spec.find_all("li")
                    for item in items:
                        text = item.text.strip()
                        if "Brand" in text:
                            brand = text.split(":")[-1].strip()
                        elif "Storage" in text:
                            storage = text.strip()
                        elif "RAM" in text or "ROM" in text:
                            ram = text.strip()
                        elif "Processor" in text:
                            processor = text.strip()
                        elif "mAh" in text:
                            battery = text.strip()
                        elif "inch" in text or "cm" in text:
                            display = text.strip()
                        elif "Camera" in text or "MP" in text:
                            camera = text.strip()
                        else:
                            feature = text.strip()

                # Append data to the list
                data.append([name, price, Ori_Price, Discount, rating, reviews, brand, storage, ram, processor, battery, feature, display, camera])

            except AttributeError:
                continue  # Skip if any required data is missing

    except requests.RequestException as e:
        print(f"Error fetching page {page_num}: {e}")
        return

In [16]:
data = []

# Scrape 5 pages (you can increase if needed)
for page in range(1, 40):  # Change 6 to 55 if you want all pages
    print(f"Scraping page {page}...")
    scrape_page(page, data)
    time.sleep(5)  # Sleep to avoid being blocked

print("Scraping completed!")

# Convert data to DataFrame
df = pd.DataFrame(data, columns=["Name", "Price", "Ori_Price", "Discount", "Rating", "Number of Reviews", "Brand", "Storage", "RAM", "Processor", "Battery", "Feature", "Display", "Camera"])

# Save DataFrame to CSV
df.to_csv("flipkart_tablets.csv", index=False)
print("Data saved to flipkart_tablets.csv")

Scraping page 1...

Fetching: https://www.flipkart.com/search?q=tablets&page=1
Response Code: 200
Scraping page 2...

Fetching: https://www.flipkart.com/search?q=tablets&page=2
Response Code: 200
Scraping page 3...

Fetching: https://www.flipkart.com/search?q=tablets&page=3
Response Code: 200
Scraping page 4...

Fetching: https://www.flipkart.com/search?q=tablets&page=4
Response Code: 200
Scraping page 5...

Fetching: https://www.flipkart.com/search?q=tablets&page=5
Response Code: 200
Scraping page 6...

Fetching: https://www.flipkart.com/search?q=tablets&page=6
Response Code: 200
Scraping page 7...

Fetching: https://www.flipkart.com/search?q=tablets&page=7
Response Code: 200
Scraping page 8...

Fetching: https://www.flipkart.com/search?q=tablets&page=8
Response Code: 200
Scraping page 9...

Fetching: https://www.flipkart.com/search?q=tablets&page=9
Response Code: 200
Scraping page 10...

Fetching: https://www.flipkart.com/search?q=tablets&page=10
Response Code: 200
Scraping page 11..

PermissionError: [Errno 13] Permission denied: 'flipkart_tablets.csv'

In [17]:
df = pd.read_csv(r"flipkart_tablets.csv")
df

,Name,Price,Ori_Price,Discount,Rating,Number of Reviews,Brand,Storage,RAM,Processor,Battery,Feature,Display,Camera
0,SAMSUNG Galaxy Tab S9 FE+ 8 GB RAM 128 GB ROM ...,"₹52,999","₹55,999",5% off,4.6,"14,715 Ratings",NaN,NaN,8 GB RAM | 128 GB ROM,Processor: Exynos 1380,NaN,1 Year Manufacturer Warranty for Device and 6 ...,31.5 cm (12.4 Inch) WQXGA Display,8.0 MP Primary Camera | 12 MP Front
1,DELL Inspiron 7440 2-in-1 Intel Core i5 13th G...,"₹71,990","₹85,574",15% off,4.0,5 Ratings,NaN,NaN,16 GB DDR5 RAM,Intel Core i5 Processor (13th Gen),NaN,1 Year Onsite Hardware Service,35.56 cm (14 Inch) Touchscreen Display,NaN
2,SAMSUNG Galaxy Tab A9+ 8 GB RAM 128 GB ROM 11....,"₹19,999","₹32,999",39% off,4.5,"23,231 Ratings",NaN,NaN,8 GB RAM | 128 GB ROM,Processor: Snapdragon 695,NaN,1 Year manufacturer warranty for device and 6 ...,27.94 cm (11.0 inch) WQXGA Display,8.0 MP Primary Camera | 5 MP Front
3,realme Pad 2 Lite 4 GB RAM 128 GB ROM 11.0 inc...,"₹12,499","₹20,999",40% off,4.4,"3,889 Ratings",NaN,NaN,4 GB RAM | 128 GB ROM | Expandable Upto 1 TB,Processor: Helio G99 Chipset,Android 13 | Battery: 8300 mAh Lithium Ion,1 Year Domestic Warranty,27.94 cm (11.0 inch) 2K Display,8 MP Primary Camera | 5 MP Front
4,REDMI Pad SE 6 GB RAM 128 GB ROM 11.0 inch wi...,"₹12,999","₹16,999",23% off,4.4,"7,108 Ratings",NaN,NaN,6 GB RAM | 128 GB ROM | Expandable Upto 1 TB,Processor: Snapdragon 680,Android 13 | Battery: 8000 mAh Lithium Ion,1 Year Domestic Warranty,27.94 cm (11.0 inch) Full HD Display,8.0 MP Primary Camera | 5 MP Front
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
979,Apple 2024 iPad Air (M2) 256 GB ROM 13.0 Inch ...,"₹87,900","₹89,900",2% off,4.7,40 Ratings,NaN,NaN,256 GB ROM,Processor: Apple M2 Chip,NaN,Apple One (1) Year Limited Warranty,33.02 cm (13.0 Inch) Display,12 MP Primary Camera | 12 MP Front
980,Apple iPad Pro 2020 (4th Generation) 6 GB RAM ...,"₹1,44,900","₹1,48,900",2% off,4.7,278 Ratings,NaN,NaN,6 GB RAM | 1 TB ROM,Processor: A12Z Bionic Chip with 64-bit Archit...,NaN,1 Year Limited Hardware Warranty,32.77 cm (12.9 inch) Quad HD Display,12 MP Primary Camera | 7 MP Front
981,Apple iPad Pro (2018) 256 GB ROM 12.9 inch wit...,"₹1,17,900",NaN,NaN,4.8,105 Ratings,NaN,NaN,256 GB ROM,"Processor: A12x Bionic Chip, 64-bit Architectu...",NaN,1 Year Manufacturer Warranty for Device and In...,32.77 cm (12.9 inch) Quad HD Display,12.0 MP Primary Camera | 7 MP Front
982,Apple iPad Pro 2020 (4th Generation) 6 GB RAM ...,"₹94,900","₹98,900",4% off,4.7,278 Ratings,NaN,NaN,6 GB RAM | 256 GB ROM,Processor: A12Z Bionic Chip with 64-bit Archit...,NaN,1 Year Limited Hardware Warranty,32.77 cm (12.9 inch) Quad HD Display,12.0 MP Primary Camera | 7 MP Front


In [18]:
import re

def extract_brand(name):
    match = re.search(r"^\b([A-Za-z]+)", str(name))  # Extract first word as Brand
    return match.group(1) if match else None  

df["Brand"] = df["Name"].apply(extract_brand)

In [19]:
import re

def extract_storage(name):
    match = re.search(r"(\d+)\s*GB\s*ROM|\b(\d)\s*TB\b", str(name))  # Look for GB or TB
    if match:
        return match.group(1) if match.group(1) else str(int(match.group(2)) * 1024)  # Convert TB to GB
    return None  

df["Storage"] = df["Name"].apply(extract_storage)

In [20]:
print(df[["Name", "Brand", "Storage"]].head(20))

                                                 Name    Brand Storage
0   SAMSUNG Galaxy Tab S9 FE+ 8 GB RAM 128 GB ROM ...  SAMSUNG     128
1   DELL Inspiron 7440 2-in-1 Intel Core i5 13th G...     DELL    None
2   SAMSUNG Galaxy Tab A9+ 8 GB RAM 128 GB ROM 11....  SAMSUNG     128
3   realme Pad 2 Lite 4 GB RAM 128 GB ROM 11.0 inc...   realme     128
4   REDMI  Pad SE 6 GB RAM 128 GB ROM 11.0 inch wi...    REDMI     128
5   Apple 2025 iPad (A16) 128 GB ROM 11.0 inch wit...    Apple     128
6   realme Pad 2 6 GB RAM 128 GB ROM 11.5 inch wit...   realme     128
7   SAMSUNG Galaxy Tab S10+ 12 GB RAM 256 GB ROM 1...  SAMSUNG     256
8   REDMI  Pad SE 4 GB RAM 128 GB ROM 11.0 inch wi...    REDMI     128
9   Lenovo Tab M11 4 GB RAM 128 GB ROM 11.0 inch w...   Lenovo     128
10  REDMI  Pad SE 4 GB RAM 128 GB ROM 11.0 inch wi...    REDMI     128
11  REDMI  Pad SE 8 GB RAM 128 GB ROM 11.0 inch wi...    REDMI     128
12  realme Pad 2 8 GB RAM 256 GB ROM 11.5 inch wit...   realme     256
13  RE

In [21]:
df

,Name,Price,Ori_Price,Discount,Rating,Number of Reviews,Brand,Storage,RAM,Processor,Battery,Feature,Display,Camera
0,SAMSUNG Galaxy Tab S9 FE+ 8 GB RAM 128 GB ROM ...,"₹52,999","₹55,999",5% off,4.6,"14,715 Ratings",SAMSUNG,128,8 GB RAM | 128 GB ROM,Processor: Exynos 1380,NaN,1 Year Manufacturer Warranty for Device and 6 ...,31.5 cm (12.4 Inch) WQXGA Display,8.0 MP Primary Camera | 12 MP Front
1,DELL Inspiron 7440 2-in-1 Intel Core i5 13th G...,"₹71,990","₹85,574",15% off,4.0,5 Ratings,DELL,None,16 GB DDR5 RAM,Intel Core i5 Processor (13th Gen),NaN,1 Year Onsite Hardware Service,35.56 cm (14 Inch) Touchscreen Display,NaN
2,SAMSUNG Galaxy Tab A9+ 8 GB RAM 128 GB ROM 11....,"₹19,999","₹32,999",39% off,4.5,"23,231 Ratings",SAMSUNG,128,8 GB RAM | 128 GB ROM,Processor: Snapdragon 695,NaN,1 Year manufacturer warranty for device and 6 ...,27.94 cm (11.0 inch) WQXGA Display,8.0 MP Primary Camera | 5 MP Front
3,realme Pad 2 Lite 4 GB RAM 128 GB ROM 11.0 inc...,"₹12,499","₹20,999",40% off,4.4,"3,889 Ratings",realme,128,4 GB RAM | 128 GB ROM | Expandable Upto 1 TB,Processor: Helio G99 Chipset,Android 13 | Battery: 8300 mAh Lithium Ion,1 Year Domestic Warranty,27.94 cm (11.0 inch) 2K Display,8 MP Primary Camera | 5 MP Front
4,REDMI Pad SE 6 GB RAM 128 GB ROM 11.0 inch wi...,"₹12,999","₹16,999",23% off,4.4,"7,108 Ratings",REDMI,128,6 GB RAM | 128 GB ROM | Expandable Upto 1 TB,Processor: Snapdragon 680,Android 13 | Battery: 8000 mAh Lithium Ion,1 Year Domestic Warranty,27.94 cm (11.0 inch) Full HD Display,8.0 MP Primary Camera | 5 MP Front
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
979,Apple 2024 iPad Air (M2) 256 GB ROM 13.0 Inch ...,"₹87,900","₹89,900",2% off,4.7,40 Ratings,Apple,256,256 GB ROM,Processor: Apple M2 Chip,NaN,Apple One (1) Year Limited Warranty,33.02 cm (13.0 Inch) Display,12 MP Primary Camera | 12 MP Front
980,Apple iPad Pro 2020 (4th Generation) 6 GB RAM ...,"₹1,44,900","₹1,48,900",2% off,4.7,278 Ratings,Apple,1024,6 GB RAM | 1 TB ROM,Processor: A12Z Bionic Chip with 64-bit Archit...,NaN,1 Year Limited Hardware Warranty,32.77 cm (12.9 inch) Quad HD Display,12 MP Primary Camera | 7 MP Front
981,Apple iPad Pro (2018) 256 GB ROM 12.9 inch wit...,"₹1,17,900",NaN,NaN,4.8,105 Ratings,Apple,256,256 GB ROM,"Processor: A12x Bionic Chip, 64-bit Architectu...",NaN,1 Year Manufacturer Warranty for Device and In...,32.77 cm (12.9 inch) Quad HD Display,12.0 MP Primary Camera | 7 MP Front
982,Apple iPad Pro 2020 (4th Generation) 6 GB RAM ...,"₹94,900","₹98,900",4% off,4.7,278 Ratings,Apple,256,6 GB RAM | 256 GB ROM,Processor: A12Z Bionic Chip with 64-bit Archit...,NaN,1 Year Limited Hardware Warranty,32.77 cm (12.9 inch) Quad HD Display,12.0 MP Primary Camera | 7 MP Front


In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 984 entries, 0 to 983
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Name               984 non-null    object 
 1   Price              983 non-null    object 
 2   Ori_Price          696 non-null    object 
 3   Discount           601 non-null    object 
 4   Rating             921 non-null    float64
 5   Number of Reviews  921 non-null    object 
 6   Brand              984 non-null    object 
 7   Storage            924 non-null    object 
 8   RAM                984 non-null    object 
 9   Processor          898 non-null    object 
 10  Battery            346 non-null    object 
 11  Feature            981 non-null    object 
 12  Display            984 non-null    object 
 13  Camera             945 non-null    object 
dtypes: float64(1), object(13)
memory usage: 107.8+ KB


In [23]:
df.isna().sum()

Name                   0
Price                  1
Ori_Price            288
Discount             383
Rating                63
Number of Reviews     63
Brand                  0
Storage               60
RAM                    0
Processor             86
Battery              638
Feature                3
Display                0
Camera                39
dtype: int64

In [24]:
import re

def extract_tablet_name(name):
    return re.split(r"\d+\s*GB\s*(RAM|ROM)", name)[0].strip()

df["Name"] = df["Name"].apply(extract_tablet_name)

In [25]:
df["Discount"] = df["Discount"].str.replace(r"\s*% off\s*", "", regex=True)

In [26]:
df["Price"] = df["Price"].str.replace(r"[₹,]", "", regex=True)
df["Ori_Price"] = df["Ori_Price"].str.replace(r"[₹,]", "", regex=True)

In [27]:
df["Price"] = df["Price"].str.replace(r"[₹,]", "", regex=True)
df["Ori_Price"] = df["Ori_Price"].str.replace(r"[₹,]", "", regex=True)

In [28]:
df["Name"]

0                              SAMSUNG Galaxy Tab S9 FE+
1      DELL Inspiron 7440 2-in-1 Intel Core i5 13th G...
2                                 SAMSUNG Galaxy Tab A9+
3                                      realme Pad 2 Lite
4                                          REDMI  Pad SE
                             ...                        
979                             Apple 2024 iPad Air (M2)
980                 Apple iPad Pro 2020 (4th Generation)
981                                Apple iPad Pro (2018)
982                 Apple iPad Pro 2020 (4th Generation)
983                                Apple iPad Pro (2018)
Name: Name, Length: 984, dtype: object

In [29]:
df

,Name,Price,Ori_Price,Discount,Rating,Number of Reviews,Brand,Storage,RAM,Processor,Battery,Feature,Display,Camera
0,SAMSUNG Galaxy Tab S9 FE+,52999,55999,5,4.6,"14,715 Ratings",SAMSUNG,128,8 GB RAM | 128 GB ROM,Processor: Exynos 1380,NaN,1 Year Manufacturer Warranty for Device and 6 ...,31.5 cm (12.4 Inch) WQXGA Display,8.0 MP Primary Camera | 12 MP Front
1,DELL Inspiron 7440 2-in-1 Intel Core i5 13th G...,71990,85574,15,4.0,5 Ratings,DELL,None,16 GB DDR5 RAM,Intel Core i5 Processor (13th Gen),NaN,1 Year Onsite Hardware Service,35.56 cm (14 Inch) Touchscreen Display,NaN
2,SAMSUNG Galaxy Tab A9+,19999,32999,39,4.5,"23,231 Ratings",SAMSUNG,128,8 GB RAM | 128 GB ROM,Processor: Snapdragon 695,NaN,1 Year manufacturer warranty for device and 6 ...,27.94 cm (11.0 inch) WQXGA Display,8.0 MP Primary Camera | 5 MP Front
3,realme Pad 2 Lite,12499,20999,40,4.4,"3,889 Ratings",realme,128,4 GB RAM | 128 GB ROM | Expandable Upto 1 TB,Processor: Helio G99 Chipset,Android 13 | Battery: 8300 mAh Lithium Ion,1 Year Domestic Warranty,27.94 cm (11.0 inch) 2K Display,8 MP Primary Camera | 5 MP Front
4,REDMI Pad SE,12999,16999,23,4.4,"7,108 Ratings",REDMI,128,6 GB RAM | 128 GB ROM | Expandable Upto 1 TB,Processor: Snapdragon 680,Android 13 | Battery: 8000 mAh Lithium Ion,1 Year Domestic Warranty,27.94 cm (11.0 inch) Full HD Display,8.0 MP Primary Camera | 5 MP Front
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
979,Apple 2024 iPad Air (M2),87900,89900,2,4.7,40 Ratings,Apple,256,256 GB ROM,Processor: Apple M2 Chip,NaN,Apple One (1) Year Limited Warranty,33.02 cm (13.0 Inch) Display,12 MP Primary Camera | 12 MP Front
980,Apple iPad Pro 2020 (4th Generation),144900,148900,2,4.7,278 Ratings,Apple,1024,6 GB RAM | 1 TB ROM,Processor: A12Z Bionic Chip with 64-bit Archit...,NaN,1 Year Limited Hardware Warranty,32.77 cm (12.9 inch) Quad HD Display,12 MP Primary Camera | 7 MP Front
981,Apple iPad Pro (2018),117900,NaN,NaN,4.8,105 Ratings,Apple,256,256 GB ROM,"Processor: A12x Bionic Chip, 64-bit Architectu...",NaN,1 Year Manufacturer Warranty for Device and In...,32.77 cm (12.9 inch) Quad HD Display,12.0 MP Primary Camera | 7 MP Front
982,Apple iPad Pro 2020 (4th Generation),94900,98900,4,4.7,278 Ratings,Apple,256,6 GB RAM | 256 GB ROM,Processor: A12Z Bionic Chip with 64-bit Archit...,NaN,1 Year Limited Hardware Warranty,32.77 cm (12.9 inch) Quad HD Display,12.0 MP Primary Camera | 7 MP Front


In [30]:
df["Number of Reviews"] = df["Number of Reviews"].str.replace(r"\D+", "", regex=True)

In [31]:
df

,Name,Price,Ori_Price,Discount,Rating,Number of Reviews,Brand,Storage,RAM,Processor,Battery,Feature,Display,Camera
0,SAMSUNG Galaxy Tab S9 FE+,52999,55999,5,4.6,14715,SAMSUNG,128,8 GB RAM | 128 GB ROM,Processor: Exynos 1380,NaN,1 Year Manufacturer Warranty for Device and 6 ...,31.5 cm (12.4 Inch) WQXGA Display,8.0 MP Primary Camera | 12 MP Front
1,DELL Inspiron 7440 2-in-1 Intel Core i5 13th G...,71990,85574,15,4.0,5,DELL,None,16 GB DDR5 RAM,Intel Core i5 Processor (13th Gen),NaN,1 Year Onsite Hardware Service,35.56 cm (14 Inch) Touchscreen Display,NaN
2,SAMSUNG Galaxy Tab A9+,19999,32999,39,4.5,23231,SAMSUNG,128,8 GB RAM | 128 GB ROM,Processor: Snapdragon 695,NaN,1 Year manufacturer warranty for device and 6 ...,27.94 cm (11.0 inch) WQXGA Display,8.0 MP Primary Camera | 5 MP Front
3,realme Pad 2 Lite,12499,20999,40,4.4,3889,realme,128,4 GB RAM | 128 GB ROM | Expandable Upto 1 TB,Processor: Helio G99 Chipset,Android 13 | Battery: 8300 mAh Lithium Ion,1 Year Domestic Warranty,27.94 cm (11.0 inch) 2K Display,8 MP Primary Camera | 5 MP Front
4,REDMI Pad SE,12999,16999,23,4.4,7108,REDMI,128,6 GB RAM | 128 GB ROM | Expandable Upto 1 TB,Processor: Snapdragon 680,Android 13 | Battery: 8000 mAh Lithium Ion,1 Year Domestic Warranty,27.94 cm (11.0 inch) Full HD Display,8.0 MP Primary Camera | 5 MP Front
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
979,Apple 2024 iPad Air (M2),87900,89900,2,4.7,40,Apple,256,256 GB ROM,Processor: Apple M2 Chip,NaN,Apple One (1) Year Limited Warranty,33.02 cm (13.0 Inch) Display,12 MP Primary Camera | 12 MP Front
980,Apple iPad Pro 2020 (4th Generation),144900,148900,2,4.7,278,Apple,1024,6 GB RAM | 1 TB ROM,Processor: A12Z Bionic Chip with 64-bit Archit...,NaN,1 Year Limited Hardware Warranty,32.77 cm (12.9 inch) Quad HD Display,12 MP Primary Camera | 7 MP Front
981,Apple iPad Pro (2018),117900,NaN,NaN,4.8,105,Apple,256,256 GB ROM,"Processor: A12x Bionic Chip, 64-bit Architectu...",NaN,1 Year Manufacturer Warranty for Device and In...,32.77 cm (12.9 inch) Quad HD Display,12.0 MP Primary Camera | 7 MP Front
982,Apple iPad Pro 2020 (4th Generation),94900,98900,4,4.7,278,Apple,256,6 GB RAM | 256 GB ROM,Processor: A12Z Bionic Chip with 64-bit Archit...,NaN,1 Year Limited Hardware Warranty,32.77 cm (12.9 inch) Quad HD Display,12.0 MP Primary Camera | 7 MP Front


In [32]:
df.isna().sum()

Name                   0
Price                  1
Ori_Price            288
Discount             383
Rating                63
Number of Reviews     63
Brand                  0
Storage               60
RAM                    0
Processor             86
Battery              638
Feature                3
Display                0
Camera                39
dtype: int64

In [33]:
# Fill numerical columns with median
df["Price"] = pd.to_numeric(df["Price"], errors="coerce").fillna(df["Price"].median())
df["Ori_Price"] = pd.to_numeric(df["Ori_Price"], errors="coerce").fillna(df["Ori_Price"].median())
df["Rating"] = df["Rating"].fillna(df["Rating"].median())

# Fill categorical columns with mode
for col in ["Discount", "Brand", "Storage", "RAM", "Processor", "Battery", "Feature", "Display", "Camera"]:
    df[col] = df[col].fillna(df[col].mode()[0])

print(df.isnull().sum())  # Check if missing values are handled

Name                  0
Price                 0
Ori_Price             0
Discount              0
Rating                0
Number of Reviews    63
Brand                 0
Storage               0
RAM                   0
Processor             0
Battery               0
Feature               0
Display               0
Camera                0
dtype: int64


In [34]:
df.fillna(df.median(numeric_only=True), inplace=True)  # Fill numeric columns with median
df.fillna(df.mode().iloc[0], inplace=True) # Fill categorical columns with mode
print(df.isnull().sum())

Name                 0
Price                0
Ori_Price            0
Discount             0
Rating               0
Number of Reviews    0
Brand                0
Storage              0
RAM                  0
Processor            0
Battery              0
Feature              0
Display              0
Camera               0
dtype: int64


In [35]:
df = df.copy()
df

,Name,Price,Ori_Price,Discount,Rating,Number of Reviews,Brand,Storage,RAM,Processor,Battery,Feature,Display,Camera
0,SAMSUNG Galaxy Tab S9 FE+,52999.0,55999.0,5,4.6,14715,SAMSUNG,128,8 GB RAM | 128 GB ROM,Processor: Exynos 1380,Android 9.0 (Pie) | Battery: 7000 mAh Lithium ...,1 Year Manufacturer Warranty for Device and 6 ...,31.5 cm (12.4 Inch) WQXGA Display,8.0 MP Primary Camera | 12 MP Front
1,DELL Inspiron 7440 2-in-1 Intel Core i5 13th G...,71990.0,85574.0,15,4.0,5,DELL,128,16 GB DDR5 RAM,Intel Core i5 Processor (13th Gen),Android 9.0 (Pie) | Battery: 7000 mAh Lithium ...,1 Year Onsite Hardware Service,35.56 cm (14 Inch) Touchscreen Display,12 MP Primary Camera | 12 MP Front
2,SAMSUNG Galaxy Tab A9+,19999.0,32999.0,39,4.5,23231,SAMSUNG,128,8 GB RAM | 128 GB ROM,Processor: Snapdragon 695,Android 9.0 (Pie) | Battery: 7000 mAh Lithium ...,1 Year manufacturer warranty for device and 6 ...,27.94 cm (11.0 inch) WQXGA Display,8.0 MP Primary Camera | 5 MP Front
3,realme Pad 2 Lite,12499.0,20999.0,40,4.4,3889,realme,128,4 GB RAM | 128 GB ROM | Expandable Upto 1 TB,Processor: Helio G99 Chipset,Android 13 | Battery: 8300 mAh Lithium Ion,1 Year Domestic Warranty,27.94 cm (11.0 inch) 2K Display,8 MP Primary Camera | 5 MP Front
4,REDMI Pad SE,12999.0,16999.0,23,4.4,7108,REDMI,128,6 GB RAM | 128 GB ROM | Expandable Upto 1 TB,Processor: Snapdragon 680,Android 13 | Battery: 8000 mAh Lithium Ion,1 Year Domestic Warranty,27.94 cm (11.0 inch) Full HD Display,8.0 MP Primary Camera | 5 MP Front
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
979,Apple 2024 iPad Air (M2),87900.0,89900.0,2,4.7,40,Apple,256,256 GB ROM,Processor: Apple M2 Chip,Android 9.0 (Pie) | Battery: 7000 mAh Lithium ...,Apple One (1) Year Limited Warranty,33.02 cm (13.0 Inch) Display,12 MP Primary Camera | 12 MP Front
980,Apple iPad Pro 2020 (4th Generation),144900.0,148900.0,2,4.7,278,Apple,1024,6 GB RAM | 1 TB ROM,Processor: A12Z Bionic Chip with 64-bit Archit...,Android 9.0 (Pie) | Battery: 7000 mAh Lithium ...,1 Year Limited Hardware Warranty,32.77 cm (12.9 inch) Quad HD Display,12 MP Primary Camera | 7 MP Front
981,Apple iPad Pro (2018),117900.0,37949.5,15,4.8,105,Apple,256,256 GB ROM,"Processor: A12x Bionic Chip, 64-bit Architectu...",Android 9.0 (Pie) | Battery: 7000 mAh Lithium ...,1 Year Manufacturer Warranty for Device and In...,32.77 cm (12.9 inch) Quad HD Display,12.0 MP Primary Camera | 7 MP Front
982,Apple iPad Pro 2020 (4th Generation),94900.0,98900.0,4,4.7,278,Apple,256,6 GB RAM | 256 GB ROM,Processor: A12Z Bionic Chip with 64-bit Archit...,Android 9.0 (Pie) | Battery: 7000 mAh Lithium ...,1 Year Limited Hardware Warranty,32.77 cm (12.9 inch) Quad HD Display,12.0 MP Primary Camera | 7 MP Front


In [36]:
print(df.duplicated().sum())  # Count duplicate rows
print(df[df.duplicated()])  # Display duplicate rows

368
                                                  Name     Price  Ori_Price  \
10                                       REDMI  Pad SE   11999.0    14999.0   
17                                       REDMI  Pad SE   13999.0    19999.0   
19                              SAMSUNG Galaxy Tab A9+   19999.0    32999.0   
23                                        realme Pad 2   18999.0    32999.0   
26   DELL Inspiron 7440 2-in-1 Intel Core i5 13th G...   71990.0    85574.0   
..                                                 ...       ...        ...   
979                           Apple 2024 iPad Air (M2)   87900.0    89900.0   
980               Apple iPad Pro 2020 (4th Generation)  144900.0   148900.0   
981                              Apple iPad Pro (2018)  117900.0    37949.5   
982               Apple iPad Pro 2020 (4th Generation)   94900.0    98900.0   
983                              Apple iPad Pro (2018)  103900.0    37949.5   

    Discount  Rating Number of Reviews    Brand

In [37]:
print(df.isna().sum())  # Count NaN values per column
print(df[df.isna().any(axis=1)])  # Display rows with NaN values

Name                 0
Price                0
Ori_Price            0
Discount             0
Rating               0
Number of Reviews    0
Brand                0
Storage              0
RAM                  0
Processor            0
Battery              0
Feature              0
Display              0
Camera               0
dtype: int64
Empty DataFrame
Columns: [Name, Price, Ori_Price, Discount, Rating, Number of Reviews, Brand, Storage, RAM, Processor, Battery, Feature, Display, Camera]
Index: []


In [38]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 984 entries, 0 to 983
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Name               984 non-null    object 
 1   Price              984 non-null    float64
 2   Ori_Price          984 non-null    float64
 3   Discount           984 non-null    object 
 4   Rating             984 non-null    float64
 5   Number of Reviews  984 non-null    object 
 6   Brand              984 non-null    object 
 7   Storage            984 non-null    object 
 8   RAM                984 non-null    object 
 9   Processor          984 non-null    object 
 10  Battery            984 non-null    object 
 11  Feature            984 non-null    object 
 12  Display            984 non-null    object 
 13  Camera             984 non-null    object 
dtypes: float64(3), object(11)
memory usage: 107.8+ KB


In [39]:
# Convert numeric columns to integers
df["Price"] = df["Price"].astype(int)
df["Ori_Price"] = df["Ori_Price"].astype(int)

# Convert storage, RAM, and display from object to integers (extracting numbers if needed)
df["Storage"] = df["Storage"].str.extract(r"(\d+)").astype(float).astype("Int64")
df["RAM"] = df["RAM"].str.extract(r"(\d+)").astype(float).astype("Int64")
df["Display"] = df["Display"].str.extract(r"(\d+)").astype(float).astype("Int64")

# Check updated data types
print(df.dtypes)

Name                  object
Price                  int32
Ori_Price              int32
Discount              object
Rating               float64
Number of Reviews     object
Brand                 object
Storage                Int64
RAM                    Int64
Processor             object
Battery               object
Feature               object
Display                Int64
Camera                object
dtype: object


In [40]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 984 entries, 0 to 983
Data columns (total 14 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Name               984 non-null    object 
 1   Price              984 non-null    int32  
 2   Ori_Price          984 non-null    int32  
 3   Discount           984 non-null    object 
 4   Rating             984 non-null    float64
 5   Number of Reviews  984 non-null    object 
 6   Brand              984 non-null    object 
 7   Storage            984 non-null    Int64  
 8   RAM                984 non-null    Int64  
 9   Processor          984 non-null    object 
 10  Battery            984 non-null    object 
 11  Feature            984 non-null    object 
 12  Display            984 non-null    Int64  
 13  Camera             984 non-null    object 
dtypes: Int64(3), float64(1), int32(2), object(8)
memory usage: 102.9+ KB


In [41]:
df

,Name,Price,Ori_Price,Discount,Rating,Number of Reviews,Brand,Storage,RAM,Processor,Battery,Feature,Display,Camera
0,SAMSUNG Galaxy Tab S9 FE+,52999,55999,5,4.6,14715,SAMSUNG,128,8,Processor: Exynos 1380,Android 9.0 (Pie) | Battery: 7000 mAh Lithium ...,1 Year Manufacturer Warranty for Device and 6 ...,31,8.0 MP Primary Camera | 12 MP Front
1,DELL Inspiron 7440 2-in-1 Intel Core i5 13th G...,71990,85574,15,4.0,5,DELL,128,16,Intel Core i5 Processor (13th Gen),Android 9.0 (Pie) | Battery: 7000 mAh Lithium ...,1 Year Onsite Hardware Service,35,12 MP Primary Camera | 12 MP Front
2,SAMSUNG Galaxy Tab A9+,19999,32999,39,4.5,23231,SAMSUNG,128,8,Processor: Snapdragon 695,Android 9.0 (Pie) | Battery: 7000 mAh Lithium ...,1 Year manufacturer warranty for device and 6 ...,27,8.0 MP Primary Camera | 5 MP Front
3,realme Pad 2 Lite,12499,20999,40,4.4,3889,realme,128,4,Processor: Helio G99 Chipset,Android 13 | Battery: 8300 mAh Lithium Ion,1 Year Domestic Warranty,27,8 MP Primary Camera | 5 MP Front
4,REDMI Pad SE,12999,16999,23,4.4,7108,REDMI,128,6,Processor: Snapdragon 680,Android 13 | Battery: 8000 mAh Lithium Ion,1 Year Domestic Warranty,27,8.0 MP Primary Camera | 5 MP Front
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
979,Apple 2024 iPad Air (M2),87900,89900,2,4.7,40,Apple,256,256,Processor: Apple M2 Chip,Android 9.0 (Pie) | Battery: 7000 mAh Lithium ...,Apple One (1) Year Limited Warranty,33,12 MP Primary Camera | 12 MP Front
980,Apple iPad Pro 2020 (4th Generation),144900,148900,2,4.7,278,Apple,1024,6,Processor: A12Z Bionic Chip with 64-bit Archit...,Android 9.0 (Pie) | Battery: 7000 mAh Lithium ...,1 Year Limited Hardware Warranty,32,12 MP Primary Camera | 7 MP Front
981,Apple iPad Pro (2018),117900,37949,15,4.8,105,Apple,256,256,"Processor: A12x Bionic Chip, 64-bit Architectu...",Android 9.0 (Pie) | Battery: 7000 mAh Lithium ...,1 Year Manufacturer Warranty for Device and In...,32,12.0 MP Primary Camera | 7 MP Front
982,Apple iPad Pro 2020 (4th Generation),94900,98900,4,4.7,278,Apple,256,6,Processor: A12Z Bionic Chip with 64-bit Archit...,Android 9.0 (Pie) | Battery: 7000 mAh Lithium ...,1 Year Limited Hardware Warranty,32,12.0 MP Primary Camera | 7 MP Front


In [42]:
df = df.drop_duplicates()

In [43]:
df

,Name,Price,Ori_Price,Discount,Rating,Number of Reviews,Brand,Storage,RAM,Processor,Battery,Feature,Display,Camera
0,SAMSUNG Galaxy Tab S9 FE+,52999,55999,5,4.6,14715,SAMSUNG,128,8,Processor: Exynos 1380,Android 9.0 (Pie) | Battery: 7000 mAh Lithium ...,1 Year Manufacturer Warranty for Device and 6 ...,31,8.0 MP Primary Camera | 12 MP Front
1,DELL Inspiron 7440 2-in-1 Intel Core i5 13th G...,71990,85574,15,4.0,5,DELL,128,16,Intel Core i5 Processor (13th Gen),Android 9.0 (Pie) | Battery: 7000 mAh Lithium ...,1 Year Onsite Hardware Service,35,12 MP Primary Camera | 12 MP Front
2,SAMSUNG Galaxy Tab A9+,19999,32999,39,4.5,23231,SAMSUNG,128,8,Processor: Snapdragon 695,Android 9.0 (Pie) | Battery: 7000 mAh Lithium ...,1 Year manufacturer warranty for device and 6 ...,27,8.0 MP Primary Camera | 5 MP Front
3,realme Pad 2 Lite,12499,20999,40,4.4,3889,realme,128,4,Processor: Helio G99 Chipset,Android 13 | Battery: 8300 mAh Lithium Ion,1 Year Domestic Warranty,27,8 MP Primary Camera | 5 MP Front
4,REDMI Pad SE,12999,16999,23,4.4,7108,REDMI,128,6,Processor: Snapdragon 680,Android 13 | Battery: 8000 mAh Lithium Ion,1 Year Domestic Warranty,27,8.0 MP Primary Camera | 5 MP Front
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
974,Apple iPad Pro (2018),89900,37949,15,4.8,105,Apple,64,64,"Processor: A12x Bionic Chip, 64-bit Architectu...",Android 9.0 (Pie) | Battery: 7000 mAh Lithium ...,1 Year Manufacturer Warranty for Device and In...,32,12.0 MP Primary Camera | 7 MP Front
975,Apple iPad Pro 2021 (5th Generation),122900,37949,15,4.6,224,Apple,256,8,Processor: Apple M1 Chip,Android 9.0 (Pie) | Battery: 7000 mAh Lithium ...,1 Year Warranty,32,12 MP Primary Camera | 12 MP Front
976,Apple iPad Pro (2018) 1 TB ROM 12.9 inch with ...,171900,37949,15,4.8,105,Apple,1024,1,"Processor: A12x Bionic Chip, 64-bit Architectu...",Android 9.0 (Pie) | Battery: 7000 mAh Lithium ...,1 Year Manufacturer Warranty for Device and In...,32,12.0 MP Primary Camera | 7 MP Front
977,Apple 2024 iPad Air (M2),76900,79900,3,4.7,40,Apple,128,128,Processor: Apple M2 Chip,Android 9.0 (Pie) | Battery: 7000 mAh Lithium ...,Apple One (1) Year Limited Warranty,33,12 MP Primary Camera | 12 MP Front


In [44]:
print(df.duplicated().sum())  # Count duplicate rows
print(df[df.duplicated()])  # Display duplicate rows

0
Empty DataFrame
Columns: [Name, Price, Ori_Price, Discount, Rating, Number of Reviews, Brand, Storage, RAM, Processor, Battery, Feature, Display, Camera]
Index: []


In [45]:
df.to_csv("Final_File")

In [46]:
df = pd.read_csv(r"Final_File")
df

,Unnamed: 0,Name,Price,Ori_Price,Discount,Rating,Number of Reviews,Brand,Storage,RAM,Processor,Battery,Feature,Display,Camera
0,0,SAMSUNG Galaxy Tab S9 FE+,52999,55999,5,4.6,14715,SAMSUNG,128,8,Processor: Exynos 1380,Android 9.0 (Pie) | Battery: 7000 mAh Lithium ...,1 Year Manufacturer Warranty for Device and 6 ...,31,8.0 MP Primary Camera | 12 MP Front
1,1,DELL Inspiron 7440 2-in-1 Intel Core i5 13th G...,71990,85574,15,4.0,5,DELL,128,16,Intel Core i5 Processor (13th Gen),Android 9.0 (Pie) | Battery: 7000 mAh Lithium ...,1 Year Onsite Hardware Service,35,12 MP Primary Camera | 12 MP Front
2,2,SAMSUNG Galaxy Tab A9+,19999,32999,39,4.5,23231,SAMSUNG,128,8,Processor: Snapdragon 695,Android 9.0 (Pie) | Battery: 7000 mAh Lithium ...,1 Year manufacturer warranty for device and 6 ...,27,8.0 MP Primary Camera | 5 MP Front
3,3,realme Pad 2 Lite,12499,20999,40,4.4,3889,realme,128,4,Processor: Helio G99 Chipset,Android 13 | Battery: 8300 mAh Lithium Ion,1 Year Domestic Warranty,27,8 MP Primary Camera | 5 MP Front
4,4,REDMI Pad SE,12999,16999,23,4.4,7108,REDMI,128,6,Processor: Snapdragon 680,Android 13 | Battery: 8000 mAh Lithium Ion,1 Year Domestic Warranty,27,8.0 MP Primary Camera | 5 MP Front
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
607,974,Apple iPad Pro (2018),89900,37949,15,4.8,105,Apple,64,64,"Processor: A12x Bionic Chip, 64-bit Architectu...",Android 9.0 (Pie) | Battery: 7000 mAh Lithium ...,1 Year Manufacturer Warranty for Device and In...,32,12.0 MP Primary Camera | 7 MP Front
608,975,Apple iPad Pro 2021 (5th Generation),122900,37949,15,4.6,224,Apple,256,8,Processor: Apple M1 Chip,Android 9.0 (Pie) | Battery: 7000 mAh Lithium ...,1 Year Warranty,32,12 MP Primary Camera | 12 MP Front
609,976,Apple iPad Pro (2018) 1 TB ROM 12.9 inch with ...,171900,37949,15,4.8,105,Apple,1024,1,"Processor: A12x Bionic Chip, 64-bit Architectu...",Android 9.0 (Pie) | Battery: 7000 mAh Lithium ...,1 Year Manufacturer Warranty for Device and In...,32,12.0 MP Primary Camera | 7 MP Front
610,977,Apple 2024 iPad Air (M2),76900,79900,3,4.7,40,Apple,128,128,Processor: Apple M2 Chip,Android 9.0 (Pie) | Battery: 7000 mAh Lithium ...,Apple One (1) Year Limited Warranty,33,12 MP Primary Camera | 12 MP Front


In [47]:
# Check if "Dell" is present in the Brand column
dell_tablets = df[df["Brand"].str.contains("Dell", case=False, na=False)]

# Display the result
print(dell_tablets)

   Unnamed: 0                                               Name  Price  \
1           1  DELL Inspiron 7440 2-in-1 Intel Core i5 13th G...  71990   

   Ori_Price  Discount  Rating  Number of Reviews Brand  Storage  RAM  \
1      85574        15     4.0                  5  DELL      128   16   

                            Processor  \
1  Intel Core i5 Processor (13th Gen)   

                                             Battery  \
1  Android 9.0 (Pie) | Battery: 7000 mAh Lithium ...   

                          Feature  Display                              Camera  
1  1 Year Onsite Hardware Service       35  12 MP Primary Camera | 12 MP Front  


In [48]:
print("Number of Dell tablets:", dell_tablets.shape[0])

Number of Dell tablets: 1


In [49]:
# Remove rows where Brand is "Dell"
df = df[~df["Brand"].str.contains("Dell", case=False, na=False)]

# Reset the index after removal
df.reset_index(drop=True, inplace=True)

# Check if Dell is removed
print("Dell tablets removed. Updated dataset:")
print(df)

Dell tablets removed. Updated dataset:
     Unnamed: 0                                               Name   Price  \
0             0                          SAMSUNG Galaxy Tab S9 FE+   52999   
1             2                             SAMSUNG Galaxy Tab A9+   19999   
2             3                                  realme Pad 2 Lite   12499   
3             4                                      REDMI  Pad SE   12999   
4             5                              Apple 2025 iPad (A16)   34900   
..          ...                                                ...     ...   
606         974                              Apple iPad Pro (2018)   89900   
607         975               Apple iPad Pro 2021 (5th Generation)  122900   
608         976  Apple iPad Pro (2018) 1 TB ROM 12.9 inch with ...  171900   
609         977                           Apple 2024 iPad Air (M2)   76900   
610         978  Wishtel IRA T1029 Dual SIM 8000mAh & Accessori...   25759   

     Ori_Price  Discount

In [50]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 611 entries, 0 to 610
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         611 non-null    int64  
 1   Name               611 non-null    object 
 2   Price              611 non-null    int64  
 3   Ori_Price          611 non-null    int64  
 4   Discount           611 non-null    int64  
 5   Rating             611 non-null    float64
 6   Number of Reviews  611 non-null    int64  
 7   Brand              611 non-null    object 
 8   Storage            611 non-null    int64  
 9   RAM                611 non-null    int64  
 10  Processor          611 non-null    object 
 11  Battery            611 non-null    object 
 12  Feature            611 non-null    object 
 13  Display            611 non-null    int64  
 14  Camera             611 non-null    object 
dtypes: float64(1), int64(8), object(6)
memory usage: 71.7+ KB


In [51]:
df.to_csv("Final_File.csv")

In [52]:
df = pd.read_csv(r"Final_File")
df

,Unnamed: 0,Name,Price,Ori_Price,Discount,Rating,Number of Reviews,Brand,Storage,RAM,Processor,Battery,Feature,Display,Camera
0,0,SAMSUNG Galaxy Tab S9 FE+,52999,55999,5,4.6,14715,SAMSUNG,128,8,Processor: Exynos 1380,Android 9.0 (Pie) | Battery: 7000 mAh Lithium ...,1 Year Manufacturer Warranty for Device and 6 ...,31,8.0 MP Primary Camera | 12 MP Front
1,1,DELL Inspiron 7440 2-in-1 Intel Core i5 13th G...,71990,85574,15,4.0,5,DELL,128,16,Intel Core i5 Processor (13th Gen),Android 9.0 (Pie) | Battery: 7000 mAh Lithium ...,1 Year Onsite Hardware Service,35,12 MP Primary Camera | 12 MP Front
2,2,SAMSUNG Galaxy Tab A9+,19999,32999,39,4.5,23231,SAMSUNG,128,8,Processor: Snapdragon 695,Android 9.0 (Pie) | Battery: 7000 mAh Lithium ...,1 Year manufacturer warranty for device and 6 ...,27,8.0 MP Primary Camera | 5 MP Front
3,3,realme Pad 2 Lite,12499,20999,40,4.4,3889,realme,128,4,Processor: Helio G99 Chipset,Android 13 | Battery: 8300 mAh Lithium Ion,1 Year Domestic Warranty,27,8 MP Primary Camera | 5 MP Front
4,4,REDMI Pad SE,12999,16999,23,4.4,7108,REDMI,128,6,Processor: Snapdragon 680,Android 13 | Battery: 8000 mAh Lithium Ion,1 Year Domestic Warranty,27,8.0 MP Primary Camera | 5 MP Front
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
607,974,Apple iPad Pro (2018),89900,37949,15,4.8,105,Apple,64,64,"Processor: A12x Bionic Chip, 64-bit Architectu...",Android 9.0 (Pie) | Battery: 7000 mAh Lithium ...,1 Year Manufacturer Warranty for Device and In...,32,12.0 MP Primary Camera | 7 MP Front
608,975,Apple iPad Pro 2021 (5th Generation),122900,37949,15,4.6,224,Apple,256,8,Processor: Apple M1 Chip,Android 9.0 (Pie) | Battery: 7000 mAh Lithium ...,1 Year Warranty,32,12 MP Primary Camera | 12 MP Front
609,976,Apple iPad Pro (2018) 1 TB ROM 12.9 inch with ...,171900,37949,15,4.8,105,Apple,1024,1,"Processor: A12x Bionic Chip, 64-bit Architectu...",Android 9.0 (Pie) | Battery: 7000 mAh Lithium ...,1 Year Manufacturer Warranty for Device and In...,32,12.0 MP Primary Camera | 7 MP Front
610,977,Apple 2024 iPad Air (M2),76900,79900,3,4.7,40,Apple,128,128,Processor: Apple M2 Chip,Android 9.0 (Pie) | Battery: 7000 mAh Lithium ...,Apple One (1) Year Limited Warranty,33,12 MP Primary Camera | 12 MP Front


In [53]:
df = df.drop(columns=[col for col in ["Unnamed: 0.1", "Unnamed: 0"] if col in df.columns])

In [55]:
df

,Name,Price,Ori_Price,Discount,Rating,Number of Reviews,Brand,Storage,RAM,Processor,Battery,Feature,Display,Camera
0,SAMSUNG Galaxy Tab S9 FE+,52999,55999,5,4.6,14715,SAMSUNG,128,8,Processor: Exynos 1380,Android 9.0 (Pie) | Battery: 7000 mAh Lithium ...,1 Year Manufacturer Warranty for Device and 6 ...,31,8.0 MP Primary Camera | 12 MP Front
1,DELL Inspiron 7440 2-in-1 Intel Core i5 13th G...,71990,85574,15,4.0,5,DELL,128,16,Intel Core i5 Processor (13th Gen),Android 9.0 (Pie) | Battery: 7000 mAh Lithium ...,1 Year Onsite Hardware Service,35,12 MP Primary Camera | 12 MP Front
2,SAMSUNG Galaxy Tab A9+,19999,32999,39,4.5,23231,SAMSUNG,128,8,Processor: Snapdragon 695,Android 9.0 (Pie) | Battery: 7000 mAh Lithium ...,1 Year manufacturer warranty for device and 6 ...,27,8.0 MP Primary Camera | 5 MP Front
3,realme Pad 2 Lite,12499,20999,40,4.4,3889,realme,128,4,Processor: Helio G99 Chipset,Android 13 | Battery: 8300 mAh Lithium Ion,1 Year Domestic Warranty,27,8 MP Primary Camera | 5 MP Front
4,REDMI Pad SE,12999,16999,23,4.4,7108,REDMI,128,6,Processor: Snapdragon 680,Android 13 | Battery: 8000 mAh Lithium Ion,1 Year Domestic Warranty,27,8.0 MP Primary Camera | 5 MP Front
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
607,Apple iPad Pro (2018),89900,37949,15,4.8,105,Apple,64,64,"Processor: A12x Bionic Chip, 64-bit Architectu...",Android 9.0 (Pie) | Battery: 7000 mAh Lithium ...,1 Year Manufacturer Warranty for Device and In...,32,12.0 MP Primary Camera | 7 MP Front
608,Apple iPad Pro 2021 (5th Generation),122900,37949,15,4.6,224,Apple,256,8,Processor: Apple M1 Chip,Android 9.0 (Pie) | Battery: 7000 mAh Lithium ...,1 Year Warranty,32,12 MP Primary Camera | 12 MP Front
609,Apple iPad Pro (2018) 1 TB ROM 12.9 inch with ...,171900,37949,15,4.8,105,Apple,1024,1,"Processor: A12x Bionic Chip, 64-bit Architectu...",Android 9.0 (Pie) | Battery: 7000 mAh Lithium ...,1 Year Manufacturer Warranty for Device and In...,32,12.0 MP Primary Camera | 7 MP Front
610,Apple 2024 iPad Air (M2),76900,79900,3,4.7,40,Apple,128,128,Processor: Apple M2 Chip,Android 9.0 (Pie) | Battery: 7000 mAh Lithium ...,Apple One (1) Year Limited Warranty,33,12 MP Primary Camera | 12 MP Front
